In [30]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

from recycle_cnn import CNN

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)
    
learning_rate = 0.001
epochs = 20
batch_size=16

In [32]:
trans = transforms.ToTensor()

train_set = torchvision.datasets.ImageFolder('data/train',transform=trans)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [33]:
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(train_loader)

In [34]:
for epoch in range(epochs):
    avg_cost = 0
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 3.19746041
[Epoch:    2] cost = 1.3773452
[Epoch:    3] cost = 1.04055285
[Epoch:    4] cost = 0.694290936
[Epoch:    5] cost = 0.410538822
[Epoch:    6] cost = 0.251765013
[Epoch:    7] cost = 0.159630001
[Epoch:    8] cost = 0.0994579196
[Epoch:    9] cost = 0.0729419068
[Epoch:   10] cost = 0.192151904
[Epoch:   11] cost = 0.0763683394
[Epoch:   12] cost = 0.0338432118
[Epoch:   13] cost = 0.0494680889
[Epoch:   14] cost = 0.0252365842
[Epoch:   15] cost = 0.341002524
[Epoch:   16] cost = 0.223571122
[Epoch:   17] cost = 0.0875653774
[Epoch:   18] cost = 0.0440470688
[Epoch:   19] cost = 0.0378776565
[Epoch:   20] cost = 0.0310469624


In [35]:
torch.save(model, 'model.pt')